In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [5]:
import os
import re
import time

import pandas as pd
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

In [8]:
def browser(url):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(executable_path='/Users/admin/Git/web-scraping/OneFootbal/chromedriver',options=option)
    driver.get(url)
    return driver


def get_html_full(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_full_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_from_a_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[2]/of-xpa-switch-match/of-match-events/div/button"
    )
    el.click()
    time.sleep(2)
    page = driver.page_source
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[3]/div[1]/div/of-xpa-switch-match/of-match-lineup-v2/section/nav/ul/li[2]/button"
    )
    el.click()
    time.sleep(2)
    second_lineup = driver.page_source
    driver.close()
    return BeautifulSoup(page), BeautifulSoup(second_lineup)



def format_field(string):
    string = str(string)
    string = string.lower()
    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace(" ", "_")
    return string


def get_info_from_event(event):
    aux_event = {}
    aux_event["event_team"] = event["class"][-1][-4:]
    aux_event["event_time"] = event.find(
        "div", class_="match-events__item-timeline"
    ).text

    if aux_event["event_time"] == " PK ":
        aux_event["event_time"] = False
        aux_event["event_type"] = "PK"
        aux_event["event_result"] = event.find("img", class_="of-image__img")["alt"]
        aux_event["event_player"] = (
            event.find("div", class_="match-events__text").find("p").text
        )
    else:
        try:
            aux_event["event_type"] = event.find(class_="match-events__icon")[
                "aria-label"
            ]
        except:
            aux_event["event_type"] = event.find("img", class_="of-image__img")["alt"]
        try:
            for i, text in enumerate(
                event.find("div", class_="match-events__text").find_all("p")
            ):
                aux_event["action_player_" + str(i + 1)] = text.text
        except:
            aux_event["action_player_1"] = event.find(
                "p", class_="match-events__text"
            ).text

    return aux_event


def get_info_from_match(page, second_lineup):
    aux_dict = {}

    ## NAMES
    aux_dict["team_name_home"] = page.find(
        "span", class_="match-score-team__name--home"
    ).text
    aux_dict["team_name_away"] = page.find(
        "span", class_="match-score-team__name--away"
    ).text

    ## GOAL
    scores = page.find("p", class_="match-score-scores").find_all("span")
    aux_dict["team_home_score"] = scores[0].text
    aux_dict["team_away_score"] = scores[2].text

    ## STATISTICS
    description = page.find_all("div", class_="match-stats__stat-description")
    for d in description:
        field = format_field(d.find_all("p")[1].text)
        aux_dict[field + "_home"] = d.find_all("p")[0].text
        aux_dict[field + "_away"] = d.find_all("p")[2].text
        
    ## PREDICTION
    predictions = page.find('ul',class_='match-prediction__buttons').find_all('li')
    aux_dict['prediction_team_home_win'] = predictions[0].text
    aux_dict['prediction_draw'] = predictions[1].text
    aux_dict['prediction_team_away_win'] = predictions[2].text

    print(len(predictions))
        
    ## LOCATION
    entries = page.find('ul',class_='match-info__entries').find_all('li')
    aux_dict["location"] = entries[-1].find('span', class_='title-8-regular match-info__entry-subtitle').text

    ## EVENTS
    events = page.find("ul", class_="match-events__list").find_all(
        "li", class_="match-events__item"
    )
    aux_dict["events_list"] = [get_info_from_event(event) for event in events]

    aux_dict["lineup_home"] = get_lineups(page)
    aux_dict["lineup_away"] = get_lineups(second_lineup)

    return aux_dict


def get_lineups(page):
    lineup = []
    for player in page.find_all("div", class_="match-lineup-v2__player"):
        player_info = player.find('p',class_='match-lineup-v2__player-name').text.split('.')
        
        lineup.append(
            {
                 "player_name": player_info[1],
                "player_number": player_info[0],
            }
        )
    return lineup



def get_penaltis(match):
    regex_rule = r"\((\d+)\)"
    aux = match.find_all(
        "span", class_="title-7-bold simple-match-card-team__score"
    )
    aux = [
        re.search(regex_rule, i.text)[0].replace("(", "").replace(")", "") for i in aux
    ]
    return aux


def get_all_matches(html_full):
    aux_dict = []
    for match in html_full.find_all("a", class_="match-card", href=True):
        aux_dict_2 = {}
        aux_dict_2["link"] = "https://onefootball.com" + match["href"]
        aux_dict_2["stage"] = match.find_previous(
            "p", class_="label simple-match-cards-list__title-deprecated"
        ).text
        try:
            aux_dict_2["date"] = match.find("time").text
            aux_dict_2["pens"] = False
            aux_dict_2["pens_home_score"] = False
            aux_dict_2["pens_away_score"] = False
        except:
            aux_dict_2["date"] = False
            aux_dict_2["pens"] = (
                match.find(
                    "span",
                    class_="title-8-medium simple-match-card__info-message simple-match-card__info-message--secondary",
                ).text
                == "(Pens)"
            )
            home, away = get_penaltis(match)
            aux_dict_2["pens_home_score"] = home
            aux_dict_2["pens_away_score"] = away

        aux_dict.append(aux_dict_2)
    return aux_dict


<IPython.core.display.Javascript object>

## Competition

In [10]:
url = "https://onefootball.com/en/competition/champions-league-5/results"  ## Competition link
page = get_html_full(url)
matchs = get_all_matches(page)
matchs[0]

{'link': 'https://onefootball.com/en/match/2203217',
 'stage': ' Final ',
 'date': ' 29.05.2021 ',
 'pens': False,
 'pens_home_score': False,
 'pens_away_score': False}

<IPython.core.display.Javascript object>

In [11]:
%%time

def merge_information(match):
    match_info = get_info_from_match(*get_html_from_a_match(match["link"]))
    return{**match, **match_info}
    
from joblib import Parallel, delayed
aux_list = Parallel(n_jobs=6, verbose=10)(delayed(merge_information)(match) for match in matchs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   27.2s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   39.1s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   59.7s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.9min


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[2]/of-xpa-switch-match/of-match-events/div/button"}
  (Session info: headless chrome=92.0.4515.131)


<IPython.core.display.Javascript object>

In [12]:
df = pd.DataFrame(aux_list).drop("link", axis=1)
df.shape

NameError: name 'aux_list' is not defined

<IPython.core.display.Javascript object>

## Manual adjustment

In [ ]:
df[df.date == False]

In [ ]:
df

In [ ]:
df.to_csv("champions_league_2020-2021_results.csv", index=False)